In [40]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from SALib.sample import saltelli
from SALib.analyze import sobol

# Define the Ishigami function
def ishigami(x):
    a = 7
    b = 0.1
    return np.sin(x[0]) + a * np.sin(x[1])**2 + b * x[2]**4 * np.sin(x[0])

# Set the number of samples
num_samples = 5000

# Generate Sobol sequences for sensitivity analysis
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-np.pi, np.pi], [-np.pi, np.pi], [-np.pi, np.pi]]
}

# Generate Sobol sequences for sensitivity analysis
sa_samples = saltelli.sample(problem, num_samples)

# Evaluate Ishigami function at the generated samples
function_values = np.array([ishigami(sample) for sample in sa_samples])

# Create polynomial features up to the desired order
order = 8
poly = PolynomialFeatures(degree=order, include_bias=False)
poly_samples = poly.fit_transform(sa_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(poly_samples, function_values, test_size=0.2, random_state=42)

# Fit a linear regression model to obtain coefficients
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Calculate model accuracy (for example, Mean Squared Error)
mse = np.mean((y_test - y_pred)**2)
print("Mean Squared Error:", mse)


predictions = model.predict(poly_samples)
# Perform Sobol sensitivity analysis
sobol_indices = sobol.analyze(problem, predictions, print_to_console=False)

# Print the results
print("First-order Sobol indices:", sobol_indices['S1'])
print("Total Sobol indices:", sobol_indices['ST'])
print("Second-order Sobol indices:")
print(sobol_indices['S2'])


/Users/vayu/anaconda3/lib/python3.7/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (5000) is equal to `2^n`.
        
  warnings.warn(msg)


Mean Squared Error: 0.004760330175609551
First-order Sobol indices: [0.31666671 0.44178805 0.00158768]
Total Sobol indices: [0.55707474 0.44101384 0.24293542]
Second-order Sobol indices:
[[            nan -5.90653648e-05  2.41574197e-01]
 [            nan             nan  2.37110201e-04]
 [            nan             nan             nan]]


In [35]:
function_values

array([3.426362  , 3.3527401 , 0.85463176, ..., 0.7979166 , 0.2301734 ,
       0.67233814])